In [91]:
import pandas as pd
import numpy as np
import re
import textdistance  # pip install textdistance, not built-in
pd.options.mode.chained_assignment = None
from collections import Counter

# Preprocesssing

In [61]:
# load data 
# we only take a random sample 

# paths to df1 (logs) and to df2 (relative popularity)
filename_df1 = "search_history.csv.gz"
filename_df2 = "query_popularity.csv.gz"

n = 84
# it is too slow to load, so for quick check don't use random sample (the one with lambda)
df1 = pd.read_csv(filename_df1, compression="gzip", nrows=100   0000)
# df1 = pd.read_csv(filename_df1, header=0, skiprows=lambda i: i % n != 0)
df2 = pd.read_csv(filename_df2, compression="gzip")

In [62]:
# to get rid of all the fun (remove emoji characters from data)
# taken from https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b

def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [63]:
# there is a user appearing too often (might be someone from WB to check results for sanity, but anyway), we remove 'em
df1 = df1[df1["wbuser_id"] != "70311ec9008a31f743c164e6f1198c86"]


# wipe out all the unicode chars
df1["no_unicode"] = df1["UQ"].apply(remove_emoji)

# remove empty strings
df1["symbol_len"] = df1["no_unicode"].apply(lambda x: len(str(x)))
df1 = df1[df1["symbol_len"] > 0]

# remove too large and too small texts
df1 = df1[(df1["symbol_len"] <= 47) & (df1["symbol_len"] >= 4)]  # the logic is, 99% of data lies below symbol_len == 47, and 99% of data lies above symbol_len == 4

# all_words to lower case
df1["no_unicode"] = df1["no_unicode"].apply(lambda x: str(x).lower())
df2["query"] = df2["query"].apply(lambda x: str(x).lower())

# Build prefix trie and merge df1 and df2

## Merging df1 and df2

In [64]:
# generate direct index 

direct_table = df1[["no_unicode", "cnt"]].groupby(by="no_unicode").count() 
direct_table["no_unicode"] = direct_table.index
direct_table = direct_table.set_index(np.arange(0, direct_table.shape[0]))
direct_table = direct_table.sort_values(by="cnt", ascending=False)
direct_table = direct_table.set_index(np.arange(0, direct_table.shape[0]))

# merge direct_table (direct_index) with df2 
# Here, join of df1 and df2 may be implemented in a fuzzy way (e.g., intersection of n_grams/textdistance between texts < t, etc.), but we just use a direct approach for simplicity

direct_table = direct_table.rename(columns={"no_unicode": "query"})
df2_new = df2.set_index("query")
direct_index = direct_table.join(df2_new, on="query", how="inner") 
# remove duplicates
direct_index = direct_index.drop_duplicates("query")
direct_index.head()

,cnt,query,query_popularity
0,3487,куртка женская осенняя,10
1,2297,ботинки женские,10
2,2131,кроссовки женские,10
3,1449,сумка женская,10
4,1425,платье,10


## Generating prefix trie and adding spellchecker

### Do not try to use this trie on a large dataset
We implmented this trie just to be fair, but to show the results we use pandas' dataframes, although all the logic of interaction with prefix trie is preserved
It is done the way it is done, because we didn't have enough time to build the trie (even from a logs' subset of size 1 million)

In [65]:
class Node:
    def __init__(self):
        self.children = {}
        self.endOfWord = False

def insertWord(root, word):
  '''
  Loop through characters in a word and keep adding them at a new node, linking them together
  If char already in node, pass
  Increment the current to the child with the character
  After the characters in word are over, mark current as EOW
  '''
  current = root
  letter = ""
  for char in word:
    if char in current.children.keys():
      letter += char
    else:
      current.children[char] = [Node(), None, letter]
      letter += char
    current = current.children[char][0]
  current.endOfWord = True

def allWords(prefix, node, results):
  '''
  Recursively call the loop
  Prefix will be prefix + current character
  Node will be position of char's child
  results are passed by reference to keep storing result

  Eventually, when we reach EOW, the prefix will have all the chars from starting and will be the word that we need. We add this word to the result
  '''
  if type(node) is list:
    node = node[0]

  if node.endOfWord:
    results.append(prefix)
    
  for char in node.children.keys():
    #print char, node, node.children
    allWords(prefix + char, node.children[char], results)

def getWordsWithPrefix(prefix, node, prefix_result):
  '''
  We loop through charcters in the prefix along with trie
  If mismatch, return
  If no mismatch during iteration, we have reached the end of prefix. Now we need to get words from current to end with the previx that we passed. So call allWords with prefix
  '''
  current = node
  for char in prefix:
    if char in current.children.keys():
      pass
    else:
      return
    current = current.children[char][0]
  allWords(prefix, current, prefix_result)


def linkWithDirectIndex(root, direct_index_df):
  '''
  Link prefix trie and direct index, placing ids of texts in each node
  '''

  for prefix in root.children:
    escaped_prefix = ".*" + re.escape(root.children[prefix][2] + prefix) + ".*"
    ids = np.array(direct_index_df[direct_index_df["no_unicode"].apply(lambda x: re.match(escaped_prefix, x) is not None)].index)
    root.children[prefix][1] = ids

    linkWithDirectIndex(root.children[prefix][0], direct_index_df)
  
  return root

def get_texts_from_word(node, word):
  # find ids in direct index for a given word (prefix)
  current = node
  pointer = None
  for char in word:
    if char in current.children.keys():
      pointer = current.children[char][1]
      current = current.children[char][0]
    else:
      # TODO: implement speller here
      return  pointer
  return pointer

def get_texts_from_query(node, query):
  # for each word (word - entity with no spaces on the left and right) in a searched text, find candidates from direct_index
  words = query.split(" ")
  pointers = []
  for word in words:
    pointers.append(get_texts_from_word(node, word))
  return pointers

def intersection_pointers(pointers, k=100):
  """
  Return the intersection of ids (pointers are lists containing ids of texts in direct_index, pointers are linked to a prefix)
  """

  resulting_set = set()
  for pointer in pointers:
    for text_id in pointer:
      if text_id not in resulting_set:
        resulting_set.add(text_id)
      if resulting_set.__len__() > k:
        return resulting_set
  return resulting_set


In [92]:
def words(text): 
    return re.findall(r'[а-я]+|[a-z]+', text.lower())

WORDS = Counter(words(open('fulltext.txt', encoding="utf8").read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюяabcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

################ Test Code 

def unit_tests():
    assert correction('speling') == 'spelling'              # insert
    assert correction('korrectud') == 'corrected'           # replace 2
    assert correction('bycycle') == 'bicycle'               # replace
    assert correction('inconvient') == 'inconvenient'       # insert 2
    assert correction('arrainged') == 'arranged'            # delete
    assert correction('peotry') =='poetry'                  # transpose
    assert correction('peotryy') =='poetry'                 # transpose + delete
    assert correction('word') == 'word'                     # known
    assert correction('quintessential') == 'quintessential' # unknown
    assert words('This is a TEST.') == ['this', 'is', 'a', 'test']
    assert correction('человк') == 'человек'
    assert correction('мжчина') == 'мужчина'
    assert correction('жнская') == 'женская'
    assert correction('зимнння') == 'зимняя'
    assert correction('споги') == 'сапоги'
    assert correction('наушнии') == 'наушники'
    assert correction('крассовки') == 'кроссовки'
    assert correction('кошылек') == 'кошелек'
    return 'unit_tests pass'

### Merge relative (from df2) and absolute (from df1, "cnt" column) popularity of logs

In [66]:
direct_index_merged = direct_index.copy()
direct_index_merged["combined_score"] = direct_index_merged["cnt"] * direct_index_merged["query_popularity"]
direct_index_merged.head()

,cnt,query,query_popularity,combined_score
0,3487,куртка женская осенняя,10,34870
1,2297,ботинки женские,10,22970
2,2131,кроссовки женские,10,21310
3,1449,сумка женская,10,14490
4,1425,платье,10,14250


This approach works just fine, because what we're trying to do is to maximize both:

* Probability of our suggest to be what the user is looking for (the best guess - is to find the most frequent text containing our prefix), that is on account of 'cnt'
* Revenue of WB, that is on account of 'query_popularity', because it contains info about products' profitability (profitability of those products that were shown to users as a result of a given search 'query')

In [67]:
# remove cnt, we don't need it anymore
direct_index_merged = direct_index_merged[["query", "combined_score"]]

## Demonstration

Now, what comes next is, essentially, what we'll show during pitches - examples of suggests based on a given search 
Again,
* the data used is just a sample from all the data
* when we merged df1 and df2, we didn't make it fuzzy (it is too greedy - read as "slow"), but further implementation of fuzzy join may improve results!

In [70]:
def heterogeneous_suggest(current_words, top_30df, discarded, kept):
    weak, strong = None, None
    if current_words:
        for word1 in current_words:
            for word2 in current_words:
                if word1 != word2:
                    if set(word1.split(" ")).symmetric_difference(set(word2.split(" "))).__len__() == 1:
                        weight1 = top_30df[top_30df["query"] == word1]["combined_and_jaro"].iloc[0]
                        weight2 = top_30df[top_30df["query"] == word2]["combined_and_jaro"].iloc[0]
                        if weight1 > weight2:
                            weak, strong = word2, word1
                        else:
                            weak, strong = word1, word2
                        break
            if strong is None:
                strong = word1
            break
        index_strong = current_words.index(strong)
        kept.append(current_words.pop(index_strong))
        if weak is not None:
            index_weak = current_words.index(weak)
            discarded.append(current_words.pop(index_weak))
        
        if len(discarded) > 20:
            kept.append(discarded.pop(0))
            return kept
        else:
            heterogeneous_suggest(current_words, top_30df, discarded, kept)
            return kept
    else:
        return kept


def get_suggest(searched_prefix):
    result_local = direct_index_merged[direct_index_merged["query"].apply(lambda x: re.match(".*"+ searched_prefix +".*", str(x)) is not None)]  # look for a prefix among all the words
    query = result_local["query"]  # all the texts containig searched_prefix
    results = tuple(map(lambda x: textdistance.jaro_winkler(searched_prefix, x), np.array(query)))  # we use jaro_winkler text distance as an approximation of sort by words (example here - https://habr.com/ru/company/vk/blog/267469/)
    # jaro_winkler has all the properties we need as it respects the words' order and is generally rather a stable metric when it comes to legnth of a word (or texts)


    result_local["jaro_winkler"] = results
    result_local = result_local[result_local["jaro_winkler"] != 1]  # we don't really need the exact word we typed (say, I searched for a "car", why would you suggest me "car" as it doesn't add any info?)


    result = result_local.sort_values(by=["jaro_winkler", "combined_score"], ascending=False)
    result_local["combined_and_jaro"] = result_local["combined_score"] * result_local["jaro_winkler"].apply(lambda x: 5*x -4)  # here, it is a penalty for our jaro_winkler ditance. 
    # This function (namely, slope and intercept) is a hyper-parameter, which basically helps us make changes in jaro_winkler more extreme - it improves the quality of results, but again, it is a hyper-parameter

    top30df = result_local[["query", "combined_and_jaro"]].sort_values(by="combined_and_jaro", ascending=False).head(30)  # we take first 30 rows to further get rid of results which are like: samsung galaxy a34 | samsung a34
    # Which is better? we use a pretty straightforward approach. Since our dataframe is already sorted by popularity of texts and by value the search gives to the business, we just need to take the one that maximazes them! (since we combined them into a one number, \
    # we need to leave just one maximal (by "combined_and_jaro") suggest!

    kept = set(heterogeneous_suggest(list(top30df["query"]), top30df, [], []))
    return top30df[top30df["query"].apply(lambda x: x in kept)].head(10)


In [75]:
# example 1
get_suggest("куртка муж")

,query,combined_and_jaro
29,куртка мужская,4700.000000
57,куртка мужская осенняя утепленная,1221.212121
948,куртка мужская демисезонная,214.814815
846,куртка мужская зимняя с капюшоном,190.909091
735,куртка мужская демисезонная с капюшоном,182.051282
1115,куртка мужская осенняя с капюшоном,147.058824
2068,куртка мужская утепленная,124.000000
5605,куртка мужская adidas,66.666667
5931,куртка мужская демисезон,54.166667
3903,куртка мужская из натуральной кожи,52.941176


In [100]:
# example 2
get_suggest(correction("pihone"))

,query,combined_and_jaro
1374,iphone 11,280.000000
5030,iphone 12,100.000000
12879,iphone 7,42.000000
7088,iphone 6s чехол,38.400000
12228,iphone 11 чехлы,32.000000
12841,iphone x,31.500000
12663,iphone 12 pro max,24.705882
35109,iphone xr,20.000000
31615,iphone 11 128,18.461538
35156,iphone se 2020,12.857143


In [93]:
# example 3
get_suggest(correction("шарррики"))

,query,combined_and_jaro
1799,шарики на день рождения,70.956522
11167,шарики воздушные для праздника,16.000000
23640,шарики для манежа,10.588235
14889,шарики воздушные свадебные,9.692308
29776,шарики розовые,6.857143
83291,шарики холодное сердце,2.727273
121850,шарики для сухого бассейна,2.307692
122242,шарики с днем рождения,2.181818
83841,шарики буквы,2.000000
121863,шарики для настольного тенниса,1.600000
